In [110]:
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import os
import matplotlib.pyplot as plt
%matplotlib notebook 
from skimage.io import imshow, imread
from matplotlib import colors
from os import path
import ipywidgets as widgets
import sys
import math
from ipywidgets import interact, interact_manual
from sklearn.cluster import KMeans
sys.path.append(path.abspath('../MSc-CS-Project---ColourPaletteExtractor-master/'))
from colourpaletteextractor.model import model

In [59]:
foldername = ''
@interact
def choose_img_folder(folder = [f for f in os.listdir('../') if '.' not in f]):
    global foldername
    foldername = folder

interactive(children=(Dropdown(description='folder', options=('art_uk', 'LICENSE', 'MSc-CS-Project---ColourPal…

In [61]:
image_formats = ['.jpg', '.jpeg', '.gif', '.png', '.tif', '.tiff', '.bmp', '.eps',
                 '.raw', '.cr2', '.nef', '.orf', '.sr2']
image_path = ''
image = ''
@interact
def chooseimage(file = [file for file in 
                             os.listdir('../' + foldername)
                             if any(word in file for word in image_formats)]):
    global image_path, image
    image = cv2.imread("../" + foldername + "/" + file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(5, 5))
    imshow(image);
    image_path = "../" + foldername + "/" + file
    plt.show()
    recoloured_image, colour_palette, relative_frequencies = model.generate_colour_palette_from_image(image_path)

interactive(children=(Dropdown(description='file', options=('normalface1.jpg', 'vase.jpg'), value='normalface1…

In [22]:
def colour_palette_to_tuple_rgba(colour):
    templist = []
    for i in range(3):
        if colour[i] < 0:
            templist.append(256 + colour[i])
        else:
            templist.append(colour[i])
    return (templist[0] / 255, templist[1] / 255, templist[2] / 255)

def colour_palette_to_positive(colour):
    templist = []
    for i in range(3):
        if colour[i] < 0:
            templist.append(256 + colour[i])
        else:
            templist.append(colour[i])
    return (templist[0], templist[1], templist[2])

@interact
def show_color_palette_pie():
    my_labels = [colour_palette_to_positive(colour_palette[i]) for i in range(len(colour_palette))]
    plt.figure(figsize=(8, 8))
    plt.pie(relative_frequencies, labels = my_labels, 
            colors = [colour_palette_to_tuple_rgba(colour_palette[i]) for i in range(len(colour_palette))],
           autopct = '%1.0f%%',
        textprops={'fontsize': 8}
           )
    plt.title('Colour Palettes', fontsize=10)
    plt.axis('equal')
    plt.show()

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [33]:
@interact (scale_percent = (10, 100, 10))
def show_3dRGB(scale_percent = 10):
    global image
    temp_image = cv2.resize(image, None, fx=scale_percent/100, fy=scale_percent/100, interpolation = cv2.INTER_AREA) 
    
    r, g, b = cv2.split(temp_image)
    fig = plt.figure(figsize=(10, 10))
    axis = fig.add_subplot(1, 1, 1, projection="3d")
    pixel_colors = temp_image.reshape((np.shape(temp_image)[0] * np.shape(temp_image)[1], 3))
    norm = colors.Normalize(vmin=-1.,vmax=1.)
    norm.autoscale(pixel_colors)
    pixel_colors = norm(pixel_colors).tolist()
    axis.scatter(r.flatten(), g.flatten(), b.flatten(), s = 100, facecolors = pixel_colors, marker=".")
    axis.set_xlabel("Red")
    axis.set_ylabel("Green")
    axis.set_zlabel("Blue")
    
    plt.show()

interactive(children=(IntSlider(value=10, description='scale_percent', min=10, step=10), Output()), _dom_class…

In [36]:
@interact (scale_percent = (10, 100, 10))
def show_3dHSV(scale_percent = 10):
    hsv_img = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    
    temp_image = cv2.resize(image, None, fx=scale_percent/100, fy=scale_percent/100, interpolation = cv2.INTER_AREA) 
    hsv_img = cv2.resize(hsv_img, None, fx=scale_percent/100, fy=scale_percent/100, interpolation = cv2.INTER_AREA) 
    
    h, s, v = cv2.split(hsv_img)
    
    pixel_colors = temp_image.reshape((np.shape(temp_image)[0]*np.shape(temp_image)[1], 3))
    norm = colors.Normalize(vmin=-1.,vmax=1.)
    norm.autoscale(pixel_colors)
    pixel_colors = norm(pixel_colors).tolist()
    
    fig = plt.figure(figsize=(10, 10))
    axis = fig.add_subplot(1, 1, 1, projection="3d")
    axis.scatter(h.flatten(), s.flatten(), v.flatten(), s= 100, facecolors = pixel_colors, marker=".")
    axis.set_xlabel("Hue")
    axis.set_ylabel("Saturation")
    axis.set_zlabel("Value")
    plt.show()

interactive(children=(IntSlider(value=10, description='scale_percent', min=10, step=10), Output()), _dom_class…

In [39]:
@interact (r = (0, 255, 1), g = (0, 255,1), b = (0, 255,1))
def filterRGB_img(r=0, g=0, b = 0,
                 directory = widgets.Text(value='Rename_File_Here.jpg', disabled=False)):
    
    img_filtered = (image[:,:,0] >= r) & (image[:,:,1] >= g) & (image[:,:,2] >= b)
    plt.figure(num=None, figsize=(5, 5))
    img_new = image.copy()
    img_new[:, :, 0] = img_new[:, :, 0] * img_filtered
    img_new[:, :, 1] = img_new[:, :, 1] * img_filtered
    img_new[:, :, 2] = img_new[:, :, 2] * img_filtered
    plt.imshow(img_new);
    plt.show()
    
    def clicked(arg):
        download_img = Image.fromarray(img_new, 'RGB')
        if len(directory.split('/')) > 1:
            split = directory.split('/')
            folders = directory.replace(split[-1], '')
            Path(folders).mkdir(parents=True, exist_ok=True)
        download_img.save(directory)
        print("saved", end='\r')
    
    save_button = widgets.Button(description = 'Save image')   
    save_button.on_click(clicked)
    display(save_button)

interactive(children=(IntSlider(value=0, description='r', max=255), IntSlider(value=0, description='g', max=25…

In [40]:
@interact
def img_HSV(option = ["All", 'Hue','Saturation','Value']):
    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    if option == "All":
        hsv_list = ['Hue','Saturation','Value']
        fig, ax = plt.subplots(1, 3, figsize=(10,5), sharey = True)

        ax[0].imshow(img_hsv[:,:,0], cmap = 'hsv')
        ax[0].set_title(hsv_list[0], fontsize = 20)
        ax[0].axis('off')

        ax[1].imshow(img_hsv[:,:,1], cmap = 'Greys')
        ax[1].set_title(hsv_list[1], fontsize = 20)
        ax[1].axis('off')

        ax[2].imshow(img_hsv[:,:,2], cmap = 'gray')
        ax[2].set_title(hsv_list[2], fontsize = 20)
        ax[2].axis('off')
        
        fig.tight_layout()
        
    elif option == "Hue":
        plt.figure(num=None, figsize=(8, 6))
        plt.imshow(img_hsv[:,:,0], cmap='hsv')
        plt.colorbar()
    
    elif option == "Saturation":
        plt.figure(num=None, figsize=(8, 6))
        plt.imshow(img_hsv[:,:,1], cmap='Greys')
        plt.colorbar()
    
    elif option == "Value":
        plt.figure(num=None, figsize=(8, 6))
        plt.imshow(img_hsv[:,:,2], cmap='gray')
        plt.colorbar()
    
    plt.show()

interactive(children=(Dropdown(description='option', options=('All', 'Hue', 'Saturation', 'Value'), value='All…

In [41]:
@interact (Saturation = (0, 255, 1))
def img_HSV_filter(h = widgets.IntRangeSlider(description='Hue', value=[0, 255],
                                               min = 0, max = 255, step=1),
                    Saturation = 0,
                  directory = widgets.Text(value='Rename_File_Here.jpg', disabled=False)):
    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    lower_mask = img_hsv[:,:,0] >= int(h[0])
    upper_mask = img_hsv[:,:,0] <= int(h[1])
    saturation = img_hsv[:,:,1] >= Saturation
    mask = upper_mask * lower_mask * saturation
    red = image[:,:,0] * mask
    green = image[:,:,1] * mask
    blue = image[:,:,2] * mask
    image_masked = np.dstack((red,green,blue))
    plt.figure(num=None, figsize=(8, 8))
    plt.imshow(image_masked);
    
    plt.show()
    
    def clicked(arg):
        download_img = Image.fromarray(image_masked, 'RGB')
        if len(directory.split('/')) > 1:
            split = directory.split('/')
            folders = directory.replace(split[-1], '')
            Path(folders).mkdir(parents=True, exist_ok=True)
        download_img.save(directory)
        print("saved", end='\r')
    
    save_button = widgets.Button(description = 'Save image')   
    save_button.on_click(clicked)
    display(save_button)

interactive(children=(IntRangeSlider(value=(0, 255), description='Hue', max=255), IntSlider(value=0, descripti…

In [42]:
def image_to_df(img):
    df = pd.DataFrame([img[:,:,0].flatten(), img[:,:,1].flatten(), img[:,:,2].flatten()]).T
    df.columns = ['R','G','B']
    return df

In [143]:
@interact_manual (k = (2, 50, 1))
def img_HSV_filter(k = 2, 
                   directory = widgets.Text(value='Rename_Folder_Here/Rename_File_Here.jpg', 
                                            disabled=False,
                                           layout=widgets.Layout(height="auto", width="auto"))):
    temp_df = image_to_df(image)
    
    plt.figure(num=None, figsize=(5, 5))
    plt.imshow(image);
    
    kmeans = KMeans(n_clusters = k, random_state = 42).fit(temp_df)
    result = kmeans.labels_.reshape(image.shape[0],image.shape[1])
    
    fig, ax = plt.subplots(math.ceil(k/2), 2, figsize=(8, 5 * math.ceil(k/2)), sharey = True)

    temp_save_as = []
    
    for n in range(k):
        temp_img = image.copy()
        temp_img[:, :, 0] = temp_img[:, :, 0]*(result==[n])
        temp_img[:, :, 1] = temp_img[:, :, 1]*(result==[n])
        temp_img[:, :, 2] = temp_img[:, :, 2]*(result==[n])
        temp_save_as.append(temp_img)
        if k == 2:
            ax[n].imshow(temp_img)
            ax[n].axis('off')
        else:
            ax[math.floor(n/2), n%2].imshow(temp_img)
            ax[math.floor(n/2), n%2].axis('off')

#         plt.figure(num=None, figsize=(5, 5))
#         plt.imshow(temp_img);
    
    plt.show()
    
    def clicked(arg):
        if len(directory.split('/')) > 1:
            split = directory.split('/')
            folders = directory.replace(split[-1], '')
            Path(folders).mkdir(parents=True, exist_ok=True)
        
        for n in range(len(temp_save_as)):
            download_img = Image.fromarray(temp_save_as[n], 'RGB')
            download_img.save(directory.split('.')[0] + "_" + str(n) + ".jpg")
            
        print("saved", end='\r')
    
    save_button = widgets.Button(description = 'Save images')   
    save_button.on_click(clicked)
    display(save_button)

interactive(children=(IntSlider(value=2, description='k', max=50, min=2), Text(value='Rename_Folder_Here/Renam…